In [1]:
import pickle
import gensim
import pyLDAvis
!pip install pyLDAvis
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sebgha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib_inline\config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
with open('rev_train.pkl', 'rb') as f:
    rev_train = pickle.load(f)
with open('rev_test.pkl', 'rb') as f:
    rev_test = pickle.load(f)

In [3]:
print(len(rev_train))
mask = (rev_train['text'].str.len() > 50) & (rev_train['text'].str.len() < 200)
rev_train = rev_train.loc[mask]
print(len(rev_train))
print(len(rev_train.sentiment))

108356
108356
108356


In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
                  'would','really','like','great','service','came','got'])

In [5]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place','would','really','like','great','service','came','got']) 
def remove_stopwords(texts):
    out = [[word for word in simple_preprocess(str(doc))
            if word not in stop_words]
            for doc in texts]
    return out
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
def get_corpus(df):
    df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    
    return corpus, id2word, bigram
train_corpus, train_id2word, bigram_train = get_corpus(rev_train)


In [9]:
with open('train_id2word.pkl', 'wb') as f:
    pickle.dump(train_id2word, f)

In [17]:
from sklearn.ensemble import RandomForestClassifier
import gensim
maxf1=0
for iterations in range(20,100,30):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        lda_train = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus,num_topics=20, iterations=iterations,id2word=train_id2word,chunksize=100,workers=7, passes=50,eval_every = 1,per_word_topics=True)
        train_vecs = []
        for i in range(len(rev_train)):
            top_topics = (lda_train.get_document_topics(train_corpus[i],minimum_probability=0.0))
            topic_vec = [top_topics[i][1] for i in range(20)]
            topic_vec.extend([len(rev_train.iloc[i].text)])
            train_vecs.append(topic_vec)
        X = np.array(train_vecs)
        y = np.array(rev_train.sentiment)

        kf = KFold(5, shuffle=True, random_state=42)
        cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

        for train_ind, val_ind in kf.split(X, y):
                        # Assign CV IDX
            X_train, y_train = X[train_ind], y[train_ind]
            X_val, y_val = X[val_ind], y[val_ind]

                        # Scale Data
            scaler = StandardScaler()
            X_train_scale = scaler.fit_transform(X_train)
            X_val_scale = scaler.transform(X_val)

                        # Logisitic Regression
            lr = RandomForestClassifier().fit(X_train_scale, y_train)

            y_pred = lr.predict(X_val_scale)
            cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
            f1s=np.mean(cv_lr_f1)
            print("iterations",iterations, "f1",f1s)
            #if f1s>maxf1:
                #lda_train.save('lda_train.model')

iterations 20 f1 0.941669006713808
iterations 20 f1 0.9426714042706458
iterations 20 f1 0.9419760091839935
iterations 20 f1 0.9419785403300832
iterations 20 f1 0.9421606370926321
iterations 50 f1 0.9419624643002855
iterations 50 f1 0.9437714738165972
iterations 50 f1 0.9429675953830589
iterations 50 f1 0.942660856214878
iterations 50 f1 0.9429209555033035
iterations 80 f1 0.9403669724770641
iterations 80 f1 0.9423697076814432
iterations 80 f1 0.9419619620637066
iterations 80 f1 0.9415661670691788
iterations 80 f1 0.9416727042835781


In [19]:
from sklearn.ensemble import RandomForestClassifier
import gensim
maxf1=0
a=[0.005,0.05,0.5]
for alpha in a:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        lda_train = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus,num_topics=20, alpha=alpha,iterations=50,id2word=train_id2word,chunksize=100,workers=7, passes=50,eval_every = 1,per_word_topics=True)
        train_vecs = []
        for i in range(len(rev_train)):
            top_topics = (lda_train.get_document_topics(train_corpus[i],minimum_probability=0.0))
            topic_vec = [top_topics[i][1] for i in range(20)]
            topic_vec.extend([len(rev_train.iloc[i].text)])
            train_vecs.append(topic_vec)
        X = np.array(train_vecs)
        y = np.array(rev_train.sentiment)

        kf = KFold(5, shuffle=True, random_state=42)
        cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

        for train_ind, val_ind in kf.split(X, y):
                        # Assign CV IDX
            X_train, y_train = X[train_ind], y[train_ind]
            X_val, y_val = X[val_ind], y[val_ind]

                        # Scale Data
            scaler = StandardScaler()
            X_train_scale = scaler.fit_transform(X_train)
            X_val_scale = scaler.transform(X_val)

                        # Logisitic Regression
            lr = RandomForestClassifier().fit(X_train_scale, y_train)

            y_pred = lr.predict(X_val_scale)
            cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
        f1s=np.mean(cv_lr_f1)
        print("alpha",alpha, "f1",f1s)
            #if f1s>maxf1:
                #lda_train.save('lda_train.model')

alpha 0.005 f1 0.9408964923278829
alpha 0.05 f1 0.9388320129741189
alpha 0.5 f1 0.9490368380845944


In [20]:
from sklearn.ensemble import RandomForestClassifier
import gensim
maxf1=0
b=[0.005,0.05,0.5]
for eta in b:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        lda_train = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus,num_topics=20, eta=eta,iterations=50,id2word=train_id2word,chunksize=100,workers=7, passes=50,eval_every = 1,per_word_topics=True)
        train_vecs = []
        for i in range(len(rev_train)):
            top_topics = (lda_train.get_document_topics(train_corpus[i],minimum_probability=0.0))
            topic_vec = [top_topics[i][1] for i in range(20)]
            topic_vec.extend([len(rev_train.iloc[i].text)])
            train_vecs.append(topic_vec)
        X = np.array(train_vecs)
        y = np.array(rev_train.sentiment)

        kf = KFold(5, shuffle=True, random_state=42)
        cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

        for train_ind, val_ind in kf.split(X, y):
                        # Assign CV IDX
            X_train, y_train = X[train_ind], y[train_ind]
            X_val, y_val = X[val_ind], y[val_ind]

                        # Scale Data
            scaler = StandardScaler()
            X_train_scale = scaler.fit_transform(X_train)
            X_val_scale = scaler.transform(X_val)

                        # Logisitic Regression
            lr = RandomForestClassifier().fit(X_train_scale, y_train)

            y_pred = lr.predict(X_val_scale)
            cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
        f1s=np.mean(cv_lr_f1)
        print("eta",eta, "f1",f1s)
            #if f1s>maxf1:
                #lda_train.save('lda_train.model')

eta 0.005 f1 0.9406986581476027
eta 0.05 f1 0.9444861848108724
eta 0.5 f1 0.9600172805492122


In [21]:
from sklearn.ensemble import RandomForestClassifier
import gensim
maxf1=0
a=[10,20,30,70,100,150,250]
for num_topics in a:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        lda_train = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus,num_topics=num_topics, alpha=0.5,eta=0.5,iterations=50,id2word=train_id2word,chunksize=100,workers=7, passes=50,eval_every = 1,per_word_topics=True)
        train_vecs = []
        for i in range(len(rev_train)):
            top_topics = (lda_train.get_document_topics(train_corpus[i],minimum_probability=0.0))
            topic_vec = [top_topics[i][1] for i in range(num_topics)]
            topic_vec.extend([len(rev_train.iloc[i].text)])
            train_vecs.append(topic_vec)
        X = np.array(train_vecs)
        y = np.array(rev_train.sentiment)

        kf = KFold(5, shuffle=True, random_state=42)
        cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

        for train_ind, val_ind in kf.split(X, y):
                        # Assign CV IDX
            X_train, y_train = X[train_ind], y[train_ind]
            X_val, y_val = X[val_ind], y[val_ind]

                        # Scale Data
            scaler = StandardScaler()
            X_train_scale = scaler.fit_transform(X_train)
            X_val_scale = scaler.transform(X_val)

                        # Logisitic Regression
            lr = RandomForestClassifier().fit(X_train_scale, y_train)

            y_pred = lr.predict(X_val_scale)
            cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
        f1s=np.mean(cv_lr_f1)
        print("num_topics",num_topics, "f1",f1s)
        if f1s>maxf1:
            lda_train.save('lda_train.model')

num_topics 10 f1 0.9561402992825965
num_topics 20 f1 0.9588342730652364
num_topics 30 f1 0.9592046446853185
num_topics 70 f1 0.9615967150986364
num_topics 100 f1 0.9630671139628937
num_topics 150 f1 0.9603605112088595
num_topics 250 f1 0.9629133808307259


In [22]:
from sklearn.ensemble import RandomForestClassifier
import gensim
maxf1=0
a=[250,500]
for num_topics in a:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        lda_train = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus,num_topics=num_topics, alpha=0.5,eta=0.5,iterations=50,id2word=train_id2word,chunksize=100,workers=7, passes=50,eval_every = 1,per_word_topics=True)
        train_vecs = []
        for i in range(len(rev_train)):
            top_topics = (lda_train.get_document_topics(train_corpus[i],minimum_probability=0.0))
            topic_vec = [top_topics[i][1] for i in range(num_topics)]
            topic_vec.extend([len(rev_train.iloc[i].text)])
            train_vecs.append(topic_vec)
        X = np.array(train_vecs)
        y = np.array(rev_train.sentiment)

        kf = KFold(5, shuffle=True, random_state=42)
        cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

        for train_ind, val_ind in kf.split(X, y):
                        # Assign CV IDX
            X_train, y_train = X[train_ind], y[train_ind]
            X_val, y_val = X[val_ind], y[val_ind]

                        # Scale Data
            scaler = StandardScaler()
            X_train_scale = scaler.fit_transform(X_train)
            X_val_scale = scaler.transform(X_val)

                        # Logisitic Regression
            lr = RandomForestClassifier().fit(X_train_scale, y_train)

            y_pred = lr.predict(X_val_scale)
            cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
        f1s=np.mean(cv_lr_f1)
        print("num_topics",num_topics, "f1",f1s)
        if f1s>maxf1:
            lda_train.save('lda_train.model')

num_topics 250 f1 0.9633162588279239


KeyboardInterrupt: 